In [232]:
import pandas as pd
te_type = 'MITE'
print('Running for', te_type)

('Running for', 'MITE')


In [233]:
#TEs
params = {}
params['MITE'] = {'min_len':50,'max_len':800,'min_distance':5,'max_q':1.15,'min_q':0.85,'min_pident':90,'min_qcov':90,'file':'data/tes/blast_all.csv'}
#select which TE type you want to run

In [234]:
#read blast output
df = pd.read_csv(params[te_type]['file'], sep='\t', )
df.columns = ['qseqid','sseqid',
              'qstart','qend','sstart',
              'send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df.index))

('initial:', 863)


In [235]:
#filter by length
if(params[te_type]['min_len']):
    df = df[df.qlen > params[te_type]['min_len']]
print('Min len: ' + str(len(df.index)))    

Min len: 863


In [236]:
if(params[te_type]['max_len']):
    df = df[df.qlen < params[te_type]['max_len']]
print('Max len: ' + str(len(df.index)))    

Max len: 863


In [237]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params[te_type]['min_q'])]
print('min treshold:',len(df.index))

('min treshold:', 863)


In [238]:
df = df[((df.length / df.qlen) <= params[te_type]['max_q'])]
print('max treshold:',len(df.index))

('max treshold:', 863)


In [239]:
df = df[(df.pident >= params[te_type]['min_pident'])]
print('Min_pident: ' + str(len(df.index)))

Min_pident: 863


In [240]:
df = df[(df.qcovs >= params[te_type]['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))

Min qcov: 863


In [241]:
#re-arrange start and end
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)


In [242]:
df = df.sort_values(by=['sseqid','sstart', 'send'])
df.reset_index(inplace=True)
df = df.drop('index',axis=1)

In [243]:
df.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
0,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,43,517,77566,78050,738,487,31,14,5,442,90.76,569,594102056,93
1,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,44,515,86285,86766,726,484,32,14,5,438,90.50,569,594102056,93
2,MITE_T_10416|chr4A|373381651|373381997|TGTT|36...,1A,4,341,213862,214201,520,341,27,4,2,310,90.91,346,594102056,99
3,MITE_T_9379|chr5A|253680838|253681189|GC|364|F534,1A,12,348,213865,214203,514,339,30,2,1,307,90.56,351,594102056,99
4,MITE_T_8262|chr5A|273309274|273309638|TA|373|F468,1A,19,346,213867,214194,560,328,19,0,0,309,94.21,364,594102056,99


In [244]:
my_index = 0
indexes = []
discard = []
count = 0
curr = 0
total_len = len(df.index)
while my_index < total_len - 1:
    row = df.iloc[[my_index]]
    cond = True
    next_index = 1
    while cond:
        second_row = df.iloc[[my_index + next_index]]
        c1 = (row.iloc[0].sseqid == second_row.iloc[0].sseqid)
        c2 = (abs(second_row.iloc[0].sstart - row.iloc[0].sstart) <= params[te_type]['min_distance'])
        c3 = (abs(second_row.iloc[0].send - row.iloc[0].send) <= params[te_type]['min_distance'])
        cond =  c1 and c2 and c3
        if not cond and (c2 or c3):
            indexes.append(my_index)
            cond = True
        next_index += 1
    indexes.append(my_index)
    my_index += next_index
    #just a counter
    curr_new = int(my_index * 100 * 1.0 / (total_len * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 5 == 0:
            print(curr_new)
indexes.append(total_len - 1)

5
25
30
35
40
50
55
60
65
70
75
80
85
90
95
100


In [245]:
df = df[df.index.isin(indexes)]
df.sort_values(['sseqid', 'sstart'], inplace=True)
print('Non overlapped: ' + str(len(df.index)))

Non overlapped: 146


In [246]:
df.head()

,qseqid,sseqid,qstart,qend,sstart,send,score,length,mismatch,gaps,gapopen,nident,pident,qlen,slen,qcovs
0,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,43,517,77566,78050,738,487,31,14,5,442,90.76,569,594102056,93
2,MITE_T_10416|chr4A|373381651|373381997|TGTT|36...,1A,4,341,213862,214201,520,341,27,4,2,310,90.91,346,594102056,99
6,MITE_T_13765|chr7B|255748631|255748962|TTTT|33...,1A,4,328,213868,214192,504,325,29,0,0,296,91.08,331,594102056,100
11,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,44,515,305432,305913,716,484,34,14,5,436,90.08,569,594102056,93
13,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...,1A,43,517,325494,325978,728,487,33,14,5,440,90.35,569,594102056,93


In [247]:
filename = params[te_type]['file'] + '.filtered'
df.to_csv(filename, index=None, sep='\t')
filename

'data/tes/blast_1A.csv.first.filtered'